### Image Data Preprocessing
[Keras documentation](https://keras.io/api/preprocessing/image/)

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
import xml.etree.ElementTree as ET # to open xml file

from tensorflow.keras.utils import load_img
from tensorflow.keras import Sequential
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, MaxPooling2D

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None;

## Exploratory Data Analysis

In [25]:
labels = pd.read_csv("C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\labels.csv")
labels.head(10)

,filename,background,facade,window,door,cornice,sill,balcony,blind,deco,molding,pillar,shop
0,cmp_b0001.jpg,0,1,1,1,1,1,1,1,1,1,1,1
1,cmp_b0001.png,0,1,1,1,1,1,1,1,1,1,1,1
2,cmp_b0002.jpg,1,0,1,1,1,1,1,1,1,1,1,1
3,cmp_b0002.png,1,0,1,1,1,1,1,1,1,1,1,1
4,cmp_b0003.jpg,1,1,0,1,1,1,1,1,1,1,1,1
5,cmp_b0003.png,1,1,0,1,1,1,1,1,1,1,1,1
6,cmp_b0004.jpg,1,1,1,0,1,1,1,1,1,1,1,1
7,cmp_b0004.png,1,1,1,0,1,1,1,1,1,1,1,1
8,cmp_b0005.jpg,1,1,1,1,0,1,1,1,1,1,1,1
9,cmp_b0005.png,1,1,1,1,0,1,1,1,1,1,1,1


In [26]:
labels.shape

(756, 13)

Видим, что данные закодированы бинарно, всего 11 признаков и 756 картинок в датасете. Посмотрим, как выглядит картинка - png, xml, jpg.

In [31]:
pip install -qq lxml

Note: you may need to restart the kernel to use updated packages.


In [35]:
# xml
import lxml.etree as ET

xml_file = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\base\\cmp_b0076.xml"

# Parse the XML file
parser = ET.XMLParser(encoding="utf-8")
tree = ET.parse(xml_file, parser=parser)
root = tree.getroot()

# Extract the values of the desired elements
building = root.find('building')
print("Building ID:", building.attrib['id'])

facade = building.find('facade')
print("Facade ID:", facade.attrib['id'])

for window in facade.findall('window'):
    print("Window ID:", window.attrib['id'])
    print("Window position:", window.attrib['pos'])
    print("Window size:", window.attrib['size'])

XMLSyntaxError: Extra content at the end of the document, line 1, column 166 (file:/C:/Users/HOME/PycharmProjects/CV_BuildingAnalytics/base/cmp_b0076.xml, line 1)

In [22]:
# png
cv2.imread("C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\base\\cmp_b0242.png")

array([[[170,   0,   0],
        [170,   0,   0],
        [170,   0,   0],
        ...,
        [170,   0,   0],
        [170,   0,   0],
        [170,   0,   0]],

       [[170,   0,   0],
        [170,   0,   0],
        [170,   0,   0],
        ...,
        [170,   0,   0],
        [170,   0,   0],
        [170,   0,   0]],

       [[170,   0,   0],
        [170,   0,   0],
        [170,   0,   0],
        ...,
        [170,   0,   0],
        [170,   0,   0],
        [170,   0,   0]],

       ...,

       [[170,   0,   0],
        [170,   0,   0],
        [170,   0,   0],
        ...,
        [170,   0,   0],
        [170,   0,   0],
        [170,   0,   0]],

       [[170,   0,   0],
        [170,   0,   0],
        [170,   0,   0],
        ...,
        [170,   0,   0],
        [170,   0,   0],
        [170,   0,   0]],

       [[170,   0,   0],
        [170,   0,   0],
        [170,   0,   0],
        ...,
        [170,   0,   0],
        [170,   0,   0],
        [170,   0,   0]]

In [23]:
# jpg
cv2.imread("C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\base\\cmp_b0005.jpg")

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[ 55,  59,  60],
        [ 54,  58,  59],
        [ 53,  57,  58],
        ...,
        [ 47,  52,  53],
        [ 49,  54,  55],
        [ 49,  54,  55]],

       [[ 56,  60,  61],
        [ 55,  59,  60],
        [ 54,  58,  59],
        ...,
        [ 49,  54,  55],
        [ 46,  51,  52],
        [ 46,  51,  52]],

       [[ 57,  61,  62],
        [ 56,  60,  61],
        [ 55,  59,  60],
        ...,
        [ 49,  54,  55],
        [ 43,  48,  49],
        [ 43,  48,  49]]